In [ ]:
#avoidance controller 2 + trajectory points BT + tracker + selfdriving    ###### WITH haptic device
import time, serial

import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output
from _thread import *
import threading

interval = True
ef= open("exper4.txt","w+")


try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False
    

    

def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)


def f():
    global df

    while True :
        de=c.recv(1024).decode("utf-8") #Collect data from port and decode into  string

        try :
            de = de.replace(',','.')
            df = []
            for i in de.split(';') :
                df.append(float(i))
            
            xa = df[0]
            ya = df[1]
            za = df[2]
        except:
            print("An exception occurred") 
            
            

global df1,df

try:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
except serial.SerialException:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
    hap.close()
    print("No connected to hap")

s = socket.socket()
print('socket created ')
port = 1234
s.bind(('127.0.0.1', port)) #bind port with ip address
print('socket binded to port ')
s.listen(5)#listening for connection
print('socket listensing ... ')

c, addr = s.accept() #when port connected
print("\ngot connection from ", addr)
#valuesx = [-4.35018263e-01, -9.29421410e-04, 9.14249594e-01, 1.32684095e+00]
#valuesy = [-0.90194138,  -0.04603966, -0.43425397, -0.1343682 ]
#valuesz= [ 0.00291168,  1.00097859 ,  0.01025859, -0.83861419]





df1 = []
df = []
hand = 0.015
p = threading.Thread(target=f, args=())
p.start()
time.sleep(5)


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));
    with open('bt.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
    to = time.clock()
        
    while True:
        tic = time.clock()
        clear_output(wait=True)
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                dmin2 = data['poss_obs']['dmin2']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        try:
            
            xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 + hand
            yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 + hand
            zo_refr = df[1] - 0.794 + 0.0006 + hand
            print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

        except:
            xo_refr = 0
            yo_refr = 0
            zo_refr = 0 

        
       
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        
        ex = xo_refr-xn
        ey = yo_refr-yn
        ez = zo_refr-zn


        print("ATL in Robot : " + str(round(xo_refr,4)) + ", " + str(round(yo_refr,4)) + ", "+ str(round(xo_refr,4) ))

        print("ATL in Robot ERROR : " + str(round(ex,4)) + ", " + str(round(ey,4)) + ", "+ str(round(ez,4)) )

        distance = round(math.sqrt(ex**2 + ey**2 + ez**2),4)

        print("distance: " + str(distance))
        hx1=35
        hy1 = 10
        viv1 = 0
        viv2 = 0
        x3 =0
        y3 =0
        if distance <= dmin2+0.3:
        
            hx1=translate(distance,dmin2,dmin2+0.3 , 15,60)
            hy1=-10
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
        print('Actual Point',P)
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('bt.json', 'r') as fa:
                data = json.load(fa)
                n_points = data['general']['n']
                x_refr = data['position'+str(n_counter)]['xref']
                y_refr = data['position'+str(n_counter)]['yref']
                z_refr = data['position'+str(n_counter)]['zref']

        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin and n_r_ao > dmin2:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            viv1 = 30    #### HAP 
            viv2 = 30    #### HAP
            
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            if  teta < np.pi*kangle/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*hv[0,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*hv[1,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*hv[2,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[2,0]
        elif n_r_ao < dmin2:
            vx1 = 0
            vy1 = 0
            vz1 = 0
            rob.set_freedrive(1)
            
            while n_r_ao < dmin2+0.1:
                ef.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+"\n")
                try:

                    xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 + hand
                    yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 + hand
                    zo_refr = df[1] - 0.794 + 0.0006 + hand
                    print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

                except:
                    xo_refr = 0
                    yo_refr = 0
                    zo_refr = 0

                t = rob.get_pose()
              #  print("Transformation from base to tcp is: ", t)


                pose = rob.getl()
                xn = float(pose[0])
                yn = float(pose[1])
                zn = float(pose[2])

                P = np.zeros((3,1));
                P[0,0] = float(xn)
                P[1,0] = float(yn)
                P[2,0] = float(zn)



                P_obs = np.zeros((3,1));   
                P_obs[0,0] = float(xo_refr)
                P_obs[1,0] = float(yo_refr)
                P_obs[2,0] = float(zo_refr)

                r_ao = P_obs - P
                n_r_ao = np.linalg.norm(r_ao)
                rob.set_freedrive(1)
                viv1 = 70    #### HAP 
                viv2 = 70    #### HAP
                hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))

            rob.set_freedrive(0)
        
        
        else :
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            viv1=0
            hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
            print("sent to the hap: " + str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3))
            #####
            #rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=1);
        except:
            print('sent')

        time.sleep(0.1)

        toc = time.clock()
        print(e)
        ef.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')
        


In [ ]:
hap.close()

In [ ]:
hap.close()

In [ ]:
####################### Experiment 1
#avoidance controller 2 + trajectory points BT + tracker + selfdriving    ###### WITH haptic device
import time, serial

import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output
from _thread import *
import threading

interval = True
ef= open("exper1_Miguel_35.txt","w+")


try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False
    

    

def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)


def f():
    global df

    while True :
        de=c.recv(1024).decode("utf-8") #Collect data from port and decode into  string

        try :
            de = de.replace(',','.')
            df = []
            for i in de.split(';') :
                df.append(float(i))
            
            xa = df[0]
            ya = df[1]
            za = df[2]
        except:
            print("An exception occurred") 
            
            

global df1,df

try:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
except serial.SerialException:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
    hap.close()
    print("No connected to hap")

s = socket.socket()
print('socket created ')
port = 1234
s.bind(('127.0.0.1', port)) #bind port with ip address
print('socket binded to port ')
s.listen(5)#listening for connection
print('socket listensing ... ')

c, addr = s.accept() #when port connected
print("\ngot connection from ", addr)
#valuesx = [-4.35018263e-01, -9.29421410e-04, 9.14249594e-01, 1.32684095e+00]
#valuesy = [-0.90194138,  -0.04603966, -0.43425397, -0.1343682 ]
#valuesz= [ 0.00291168,  1.00097859 ,  0.01025859, -0.83861419]





df1 = []
df = []
hand = 0.015
stimuli = "none"
p = threading.Thread(target=f, args=())
p.start()
time.sleep(5)


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));
    with open('bt2points.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
    to = time.clock()
        
    while True:
        tic = time.clock()
        clear_output(wait=True)
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                dmin2 = data['poss_obs']['dmin2']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        try:
            
            xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 + hand
            yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 + hand
            zo_refr = df[1] - 0.794 + 0.0006 + hand
            print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

        except:
            xo_refr = 0
            yo_refr = 0
            zo_refr = 0 

        
       
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        
        ex = xo_refr-xn
        ey = yo_refr-yn
        ez = zo_refr-zn


        print("ATL in Robot : " + str(round(xo_refr,4)) + ", " + str(round(yo_refr,4)) + ", "+ str(round(xo_refr,4) ))

        print("ATL in Robot ERROR : " + str(round(ex,4)) + ", " + str(round(ey,4)) + ", "+ str(round(ez,4)) )

        distance = round(math.sqrt(ex**2 + ey**2 + ez**2),4)

        print("distance: " + str(distance))
        hx1=35
        hy1 = 10
        viv1 = 0
        viv2 = 0
        x3 =0
        y3 =0
#####################
        if distance <= dmin2+0.35:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.35, 15, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_35"
        
        '''
        if distance <= dmin2+0.3:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.3, 15, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_30"
            
        if distance <= dmin2+0.2:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.2, 15, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_20"
        
        '''
#######################      
        
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
        print('Actual Point',P)
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('bt2points.json', 'r') as fa:
                data = json.load(fa)
                n_points = data['general']['n']
                x_refr = data['position'+str(n_counter)]['xref']
                y_refr = data['position'+str(n_counter)]['yref']
                z_refr = data['position'+str(n_counter)]['zref']
                print("xref: "+ str(x_refr))
        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin and n_r_ao > dmin2:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            viv1 = 0    #### HAP 
            viv2 = 50    #### HAP
            stimuli = "collision_one"
            
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            
            if  teta < np.pi*kangle/180:
                
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*hv[0,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*hv[1,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*hv[2,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[2,0]
            
        elif n_r_ao < dmin2:
            vx1 = 0
            vy1 = 0
            vz1 = 0
            rob.set_freedrive(1)
            
            while n_r_ao < dmin2+0.1:
                ef.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
                try:

                    xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 + hand
                    yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 + hand
                    zo_refr = df[1] - 0.794 + 0.0006 + hand
                    print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

                except:
                    xo_refr = 0
                    yo_refr = 0
                    zo_refr = 0

                t = rob.get_pose()
              #  print("Transformation from base to tcp is: ", t)


                pose = rob.getl()
                xn = float(pose[0])
                yn = float(pose[1])
                zn = float(pose[2])

                P = np.zeros((3,1));
                P[0,0] = float(xn)
                P[1,0] = float(yn)
                P[2,0] = float(zn)



                P_obs = np.zeros((3,1));   
                P_obs[0,0] = float(xo_refr)
                P_obs[1,0] = float(yo_refr)
                P_obs[2,0] = float(zo_refr)

                r_ao = P_obs - P
                n_r_ao = np.linalg.norm(r_ao)
                rob.set_freedrive(1)
                viv1 = 70    #### HAP 
                viv2 = 70    #### HAP
                stimuli = "collision_one"
                hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))

            rob.set_freedrive(0)
        
        
        else :
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1

        
        try:
            viv1=0
            hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
            print("sent to the hap: " + str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3))
            #####
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=1);
        except:
            print('sent')

        time.sleep(0.1)

        toc = time.clock()
        print(e)
        ef.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')
        


In [ ]:
hap.close()

In [ ]:
####################### Experiment 1 ok
#### Two points, the user static hand, moves the hand in two different ways


#avoidance controller 2 + trajectory points BT(points,gripper) + tracker + selfdriving
import time, serial
import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output
from _thread import *
import threading

interval = True
f1w= open("exper1_Jons_35_10rounds_back2.txt","w+")



try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False
    
from urx.robotiq_two_finger_gripper import Robotiq_Two_Finger_Gripper

robotiqgrip = Robotiq_Two_Finger_Gripper(rob)
robotiqgrip.open_gripper()

p=(0.9, -0.43575, 0.36568, 3.14,0,0)    
rob.movel(p, vel=0.1, acc=0.1)

def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)



def f():
    global df

    while True :
        de=c.recv(1024).decode("utf-8") #Collect data from port and decode into  string

        try :
            de = de.replace(',','.')
            df = []
            for i in de.split(';') :
                df.append(float(i))
            
            xa = df[0]
            ya = df[1]
            za = df[2]
        except:
            print("An exception occurred") 
            
            

global df1,df

#### HAP
try:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
except serial.SerialException:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
    hap.close()
    
    print("No connected to hap")
#### HAP

s = socket.socket()
print('socket created ')
port = 1234
s.bind(('127.0.0.1', port)) #bind port with ip address
print('socket binded to port ')
s.listen(5)#listening for connection
print('socket listensing ... ')

c, addr = s.accept() #when port connected
print("\ngot connection from ", addr)
#valuesx = [-4.35018263e-01, -9.29421410e-04, 9.14249594e-01, 1.32684095e+00]
#valuesy = [-0.90194138,  -0.04603966, -0.43425397, -0.1343682 ]
#valuesz= [ 0.00291168,  1.00097859 ,  0.01025859, -0.83861419]


df1 = []
df = []

hand = 0.015
stimuli = "none"

p = threading.Thread(target=f, args=())
p.start()
time.sleep(5)


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.ones((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));
    with open('bt2pointsb.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
    to = time.clock()
        
    while True:
        tic = time.clock()
        clear_output(wait=True)
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                dmin2 = data['poss_obs']['dmin2']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        try:
            
            xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 #+ hand
            yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 #+ hand
            zo_refr = df[1] - 0.794 + 0.0006 #+ hand
            print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

        except:
            xo_refr = 0
            yo_refr = 0
            zo_refr = 0 

        
       
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        ###########################
        ex = xo_refr-xn
        ey = yo_refr-yn
        ez = zo_refr-zn


        print("ATL in Robot : " + str(round(xo_refr,4)) + ", " + str(round(yo_refr,4)) + ", "+ str(round(xo_refr,4) ))

        print("ATL in Robot ERROR : " + str(round(ex,4)) + ", " + str(round(ey,4)) + ", "+ str(round(ez,4)) )

        distance = round(math.sqrt(ex**2 + ey**2 + ez**2),4)

        print("distance: " + str(distance))
        hx1=35
        hy1 = 10
        viv1 = 0
        viv2 = 0
        x3 =0
        y3 =0
        
        ###########################
        
        
        if distance <= dmin2+0.35:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.35, 15, 60)    ##HAP
            hy1=-15                            
            stimuli = "position_35"
        
        '''
        if distance <= dmin2+0.3:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.3, 15, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_30"
            
        if distance <= dmin2+0.2:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.2, 15, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_20"
        
        '''
        #######################     
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
        print('Actual Point',P)
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('bt2pointsb.json', 'r') as fa:
                data = json.load(fa)
                n_points = data['general']['n']
                action = data['position'+str(n_counter)]['action']
                if action == "open":
                    rob.speedl((0 , 0, 0 , 0, 0, 0), acc=0.3, min_time=1);
                    time.sleep(0.1)
                    robotiqgrip.open_gripper()
                    time.sleep(0.1)
                elif action == "close":
                    rob.speedl((0 , 0, 0 , 0, 0, 0), acc=0.3, min_time=1);
                    time.sleep(0.1)
                    robotiqgrip.close_gripper()
                    time.sleep(0.1)
                elif action == "end":
                    rob.close()
                    hap.close()
                    sys.exit()
                    break
                else:
                    x_refr = data['position'+str(n_counter)]['xref']
                    y_refr = data['position'+str(n_counter)]['yref']
                    z_refr = data['position'+str(n_counter)]['zref']

        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin and n_r_ao > dmin2:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            
            viv1 = 0    #### HAP 
            viv2 = 50    #### HAP
            stimuli = "collision_one" #HAP
            
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            
            if  teta < np.pi*kangle/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*hv[0,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*hv[1,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*hv[2,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[2,0]
        
        elif n_r_ao < dmin2:
            vx1 = 0
            vy1 = 0
            vz1 = 0
            rob.set_freedrive(1)
            
            while n_r_ao < dmin2+0.1:
                f1w.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
                try:
                    xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 #+ hand
                    yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 #+ hand
                    zo_refr = df[1] - 0.794 + 0.0006 #+ hand
                    print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

                except:
                    xo_refr = 0
                    yo_refr = 0
                    zo_refr = 0

                t = rob.get_pose()
              #  print("Transformation from base to tcp is: ", t)


                pose = rob.getl()
                xn = float(pose[0])
                yn = float(pose[1])
                zn = float(pose[2])

                P = np.zeros((3,1));
                P[0,0] = float(xn)
                P[1,0] = float(yn)
                P[2,0] = float(zn)



                P_obs = np.zeros((3,1));   
                P_obs[0,0] = float(xo_refr)
                P_obs[1,0] = float(yo_refr)
                P_obs[2,0] = float(zo_refr)

                r_ao = P_obs - P
                n_r_ao = np.linalg.norm(r_ao)
                rob.set_freedrive(1)
                viv1 = 70    #### HAP 
                viv2 = 70    #### HAP
                stimuli = "collision_one"
                hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))

                
            rob.set_freedrive(0)
        
        
        else :
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            viv1=0
            hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
            print("sent to the hap: " + str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3))
            #####          
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=1);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        #print(e)
        f1w.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
           
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')

In [ ]:
############### Experiment 2  ##################  OK
#avoidance controller 2 + trajectory points BT + tracker + selfdriving    ###### WITH haptic device
import time, serial

import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output
from _thread import *
import threading

interval = True
ef= open("exper2_Fikre_25_1.txt","w+")


try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False
p=(0.9, -0.43575, 0.5, 3.14,0,0)    
rob.movel(p, vel=0.1, acc=0.1)

    

def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)


def f():
    global df

    while True :
        de=c.recv(1024).decode("utf-8") #Collect data from port and decode into  string

        try :
            de = de.replace(',','.')
            df = []
            for i in de.split(';') :
                df.append(float(i))
            
            xa = df[0]
            ya = df[1]
            za = df[2]
        except:
            print("An exception occurred") 
            
            

global df1,df

try:
    hap = serial.Serial('COM4', baudrate=115200, timeout=0.1)
except serial.SerialException:
    hap = serial.Serial('COM4', baudrate=115200, timeout=0.1)
    hap.close()
    print("No connected to hap")

s = socket.socket()
print('socket created ')
port = 1234
s.bind(('127.0.0.1', port)) #bind port with ip address
print('socket binded to port ')
s.listen(5)#listening for connection
print('socket listensing ... ')

c, addr = s.accept() #when port connected
print("\ngot connection from ", addr)
#valuesx = [-4.35018263e-01, -9.29421410e-04, 9.14249594e-01, 1.32684095e+00]
#valuesy = [-0.90194138,  -0.04603966, -0.43425397, -0.1343682 ]
#valuesz= [ 0.00291168,  1.00097859 ,  0.01025859, -0.83861419]





df1 = []
df = []
hand = 0.015
stimuli = "none"
p = threading.Thread(target=f, args=())
p.start()
time.sleep(5)


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));
    with open('bt.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
    to = time.clock()
        
    while True:
        tic = time.clock()
        clear_output(wait=True)
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                dmin2 = data['poss_obs']['dmin2']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        try:
            
            xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 + hand
            yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 + hand
            zo_refr = df[1] - 0.794 + 0.0006 + hand
            print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

        except:
            xo_refr = 0
            yo_refr = 0
            zo_refr = 0 

        
       
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        
        ex = xo_refr-xn
        ey = yo_refr-yn
        ez = zo_refr-zn


        print("ATL in Robot : " + str(round(xo_refr,4)) + ", " + str(round(yo_refr,4)) + ", "+ str(round(xo_refr,4) ))

        print("ATL in Robot ERROR : " + str(round(ex,4)) + ", " + str(round(ey,4)) + ", "+ str(round(ez,4)) )

        distance = round(math.sqrt(ex**2 + ey**2 + ez**2),4)

        print("distance: " + str(distance))
        hx1=35
        hy1 = 10
        viv1 = 0
        viv2 = 0
        x3 =0
        y3 =0
#####################
        '''
        if distance <= dmin2+0.35:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.35, 15, 60)    ##HAP
            hy1=-15                            
            stimuli = "position_35"
        
        '''
        if distance <= dmin2+0.3:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.3, 15, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_30"
        '''    
        if distance <= dmin2+0.2:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.2, 15, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_20"
        
        '''
#######################      
        
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
        print('Actual Point',P)
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('bt.json', 'r') as fa:
                data = json.load(fa)
                n_points = data['general']['n']
                x_refr = data['position'+str(n_counter)]['xref']
                y_refr = data['position'+str(n_counter)]['yref']
                z_refr = data['position'+str(n_counter)]['zref']

        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin and n_r_ao > dmin2-0.05:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            viv1 = 0    #### HAP 
            viv2 = 30    #### HAP
            stimuli = "collision_one"
            
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            '''
            if  teta < np.pi*kangle/180:
                
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*hv[0,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*hv[1,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*hv[2,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[2,0]
            '''
        elif n_r_ao < dmin2:
            vx1 = 0
            vy1 = 0
            vz1 = 0
            rob.set_freedrive(1)
            
            while n_r_ao < dmin2:#+0.1:
                ef.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
                try:

                    xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 + hand
                    yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 + hand
                    zo_refr = df[1] - 0.794 + 0.0006 + hand
                    print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

                except:
                    xo_refr = 0
                    yo_refr = 0
                    zo_refr = 0

                t = rob.get_pose()
              #  print("Transformation from base to tcp is: ", t)


                pose = rob.getl()
                xn = float(pose[0])
                yn = float(pose[1])
                zn = float(pose[2])

                P = np.zeros((3,1));
                P[0,0] = float(xn)
                P[1,0] = float(yn)
                P[2,0] = float(zn)



                P_obs = np.zeros((3,1));   
                P_obs[0,0] = float(xo_refr)
                P_obs[1,0] = float(yo_refr)
                P_obs[2,0] = float(zo_refr)

                r_ao = P_obs - P
                n_r_ao = np.linalg.norm(r_ao)
                rob.set_freedrive(1)
                viv1 = 70    #### HAP 
                viv2 = 70    #### HAP
                stimuli = "collision_two"
                hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))

            rob.set_freedrive(0)
        
        
        else :
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        '''
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1

        '''
        try:
            viv1=0
            hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
            print("sent to the hap: " + str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3))
            #####
            #rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=1);
        except:
            print('sent')

        time.sleep(0.1)

        toc = time.clock()
        print(e)
        ef.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')
        


In [ ]:
hap.close()

In [ ]:
############### Experiment 2  ##################  OK V2!!!!!!!!!!!!!!!!!!!!!!
#avoidance controller 2 + trajectory points BT + tracker + selfdriving    ###### WITH haptic device
import time, serial

import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output
from _thread import *
import threading

interval = True
ef= open("exper2_2_Miguel_25_2.txt","w+")


try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False
p=(0.9, -0.43575, 0.5, 3.14,0,0)    
rob.movel(p, vel=0.1, acc=0.1)

    

def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)


def f():
    global df

    while True :
        de=c.recv(1024).decode("utf-8") #Collect data from port and decode into  string

        try :
            de = de.replace(',','.')
            df = []
            for i in de.split(';') :
                df.append(float(i))
            
            xa = df[0]
            ya = df[1]
            za = df[2]
        except:
            print("An exception occurred") 
            
            

global df1,df

try:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
except serial.SerialException:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
    hap.close()
    print("No connected to hap")

s = socket.socket()
print('socket created ')
port = 1234
s.bind(('127.0.0.1', port)) #bind port with ip address
print('socket binded to port ')
s.listen(5)#listening for connection
print('socket listensing ... ')

c, addr = s.accept() #when port connected
print("\ngot connection from ", addr)
#valuesx = [-4.35018263e-01, -9.29421410e-04, 9.14249594e-01, 1.32684095e+00]
#valuesy = [-0.90194138,  -0.04603966, -0.43425397, -0.1343682 ]
#valuesz= [ 0.00291168,  1.00097859 ,  0.01025859, -0.83861419]





df1 = []
df = []
hand = 0.015
stimuli = "none"
p = threading.Thread(target=f, args=())
p.start()
time.sleep(5)


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));
    with open('bt.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
    to = time.clock()
        
    while True:
        tic = time.clock()
        clear_output(wait=True)
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                dmin2 = data['poss_obs']['dmin2']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        try:
            
            xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 + hand
            yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 + hand
            zo_refr = df[1] - 0.794 + 0.0006 + hand
            print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

        except:
            xo_refr = 0
            yo_refr = 0
            zo_refr = 0 

        
       
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        
        ex = xo_refr-xn
        ey = yo_refr-yn
        ez = zo_refr-zn


        print("ATL in Robot : " + str(round(xo_refr,4)) + ", " + str(round(yo_refr,4)) + ", "+ str(round(xo_refr,4) ))

        print("ATL in Robot ERROR : " + str(round(ex,4)) + ", " + str(round(ey,4)) + ", "+ str(round(ez,4)) )

        distance = round(math.sqrt(ex**2 + ey**2 + ez**2),4)

        print("distance: " + str(distance))
        hx1=35
        hy1 = 10
        viv1 = 0
        viv2 = 0
        x3 =0
        y3 =0
        dhap=0.2
#####################
        
        '''
        
        if distance <= dhap+0.05:  ##HAP
        
            hx1=translate(distance,dhap, dhap + 0.05, 10, 60)    ##HAP
            hy1=-11                            
            stimuli = "position_05"
        
        
        if distance <= dhap+0.1:  ##HAP
        
            hx1=translate(distance,dhap, dhap + 0.1, 10, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_10"
        
      
        if distance <= dhap+0.15:  ##HAP
        
            hx1=translate(distance,dhap, dhap + 0.15, 10, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_15"
           
        if distance <= dhap+0.2:  ##HAP
        
            hx1=translate(distance,dhap, dhap + 0.2, 10, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_20"
         '''  
        if distance <= dhap+0.25:  ##HAP
        
            hx1=translate(distance,dhap, dhap + 0.25, 10, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_25"
           
        
       
        if distance <= dhap:
            viv1 = 50
            viv2 = 50
            hx1 = 10
            hy1=-10
#######################      
        
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
        print('Actual Point',P)
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('bt.json', 'r') as fa:
                data = json.load(fa)
                n_points = data['general']['n']
                x_refr = data['position'+str(n_counter)]['xref']
                y_refr = data['position'+str(n_counter)]['yref']
                z_refr = data['position'+str(n_counter)]['zref']

        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin and n_r_ao > dmin2-0.05:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            #viv1 = 0    #### HAP 
            #viv2 = 30    #### HAP
            stimuli = "collision_one"
            
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            '''
            if  teta < np.pi*kangle/180:
                
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*hv[0,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*hv[1,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*hv[2,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[2,0]
            '''
        elif n_r_ao < dmin2:
            vx1 = 0
            vy1 = 0
            vz1 = 0
            rob.set_freedrive(1)
            
            while n_r_ao < dmin2:#+0.1:
                ef.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
                try:

                    xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 + hand
                    yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 + hand
                    zo_refr = df[1] - 0.794 + 0.0006 + hand
                    print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

                except:
                    xo_refr = 0
                    yo_refr = 0
                    zo_refr = 0

                t = rob.get_pose()
              #  print("Transformation from base to tcp is: ", t)


                pose = rob.getl()
                xn = float(pose[0])
                yn = float(pose[1])
                zn = float(pose[2])

                P = np.zeros((3,1));
                P[0,0] = float(xn)
                P[1,0] = float(yn)
                P[2,0] = float(zn)



                P_obs = np.zeros((3,1));   
                P_obs[0,0] = float(xo_refr)
                P_obs[1,0] = float(yo_refr)
                P_obs[2,0] = float(zo_refr)

                r_ao = P_obs - P
                n_r_ao = np.linalg.norm(r_ao)
                rob.set_freedrive(1)
                viv1 = 70    #### HAP 
                viv2 = 70    #### HAP
                stimuli = "collision_two"
                hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))

            rob.set_freedrive(0)
        
        
        else :
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        '''
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1

        '''
        try:
            #viv1=0
            hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
            print("sent to the hap: " + str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3))
            #####
            #rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=1);
        except:
            print('sent')

        time.sleep(0.1)

        toc = time.clock()
        print(e)
        ef.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')
        


In [ ]:
### start
import serial
import time

import binascii
import struct
#x1=173

x1=10
y1=-12
v1 = 0#100
v2 = 0#100
x3= 100
y3= 0
c=0
try:
    s = serial.Serial('COM6', baudrate=115200, timeout=0.1)
except serial.SerialException:
    s = serial.Serial('COM6', baudrate=115200, timeout=0.1)
    s.close()
    
    pass
    # print str(message_bytes)

while c<50:
    
    
    print(x1)
    c+=1
    s.write(bytes(str(x1) + "," + str(y1)+ "," + str(v1)+ "," + str(v2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
    time.sleep(0.1)
    x1+=1
    
while c>=50 and c<100:
    
    
    print(x1)
    c+=1
    s.write(bytes(str(x1) + "," + str(y1)+ "," + str(v1)+ "," + str(v2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
    time.sleep(0.1)
    x1-=1

x1=35
y1=10
x2=0
y2=0
x3= 0
y3= 0

s.write(bytes(str(x1) + "," + str(y1)+ "," + str(x2)+ "," + str(y2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
print(str(x1) + "," + str(y1)+ "," + str(x2)+ "," + str(y2)+ "," + str(x3)+ "," + str(y3)+ "\n")
s.close()


# 

In [ ]:
####################### Experiment 3
#### Two points, the user static hand, moves the hand in two different ways


#avoidance controller 2 + trajectory points BT(points,gripper) + tracker + selfdriving
import time, serial
import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output
from _thread import *
import threading

interval = True
f1w= open("exper3_Jonathan_Haptics.txt","w+")


try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False
    
from urx.robotiq_two_finger_gripper import Robotiq_Two_Finger_Gripper

robotiqgrip = Robotiq_Two_Finger_Gripper(rob)
robotiqgrip.open_gripper()

#p=(0.9, -0.43575, 0.36568, 3.14,0,0)    
#rob.movel(p, vel=0.1, acc=0.1)

def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)



def f():
    global df

    while True :
        de=c.recv(1024).decode("utf-8") #Collect data from port and decode into  string

        try :
            de = de.replace(',','.')
            df = []
            for i in de.split(';') :
                df.append(float(i))
            
            xa = df[0]
            ya = df[1]
            za = df[2]
        except:
            print("An exception occurred") 
            
            

global df1,df

#### HAP
try:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
except serial.SerialException:
    hap = serial.Serial('COM6', baudrate=115200, timeout=0.1)
    hap.close()
    
    print("No connected to hap")

#hap.write(bytes(str(35) + "," + str(15)+ "," + str(50)+ "," + str(50)+ "," + str(0)+ "," + str(0)+ "\n",'utf-8'))
#time.sleep(0.5)
#hap.write(bytes(str(35) + "," + str(10)+ "," + str(0)+ "," + str(0)+ "," + str(0)+ "," + str(0)+ "\n",'utf-8'))
                          
#### HAP

s = socket.socket()
print('socket created ')
port = 1234
s.bind(('127.0.0.1', port)) #bind port with ip address
print('socket binded to port ')
s.listen(5)#listening for connection
print('socket listensing ... ')

c, addr = s.accept() #when port connected
print("\ngot connection from ", addr)
#valuesx = [-4.35018263e-01, -9.29421410e-04, 9.14249594e-01, 1.32684095e+00]
#valuesy = [-0.90194138,  -0.04603966, -0.43425397, -0.1343682 ]
#valuesz= [ 0.00291168,  1.00097859 ,  0.01025859, -0.83861419]


df1 = []
df = []

hand = 0.015
stimuli = "none"

p = threading.Thread(target=f, args=())
p.start()
time.sleep(5)


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.ones((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));
    with open('bt.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
    to = time.clock()
        
    while True:
        tic = time.clock()
        clear_output(wait=True)
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                dmin2 = data['poss_obs']['dmin2']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        try:
            
            xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 #+ hand
            yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 #+ hand
            zo_refr = df[1] - 0.794 + 0.0006 #+ hand
            print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

        except:
            xo_refr = 0
            yo_refr = 0
            zo_refr = 0 

        
       
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        ###########################
        ex = xo_refr-xn
        ey = yo_refr-yn
        ez = zo_refr-zn


        print("ATL in Robot : " + str(round(xo_refr,4)) + ", " + str(round(yo_refr,4)) + ", "+ str(round(xo_refr,4) ))

        print("ATL in Robot ERROR : " + str(round(ex,4)) + ", " + str(round(ey,4)) + ", "+ str(round(ez,4)) )

        distance = round(math.sqrt(ex**2 + ey**2 + ez**2),4)

        print("distance: " + str(distance))
        hx1=35
        hy1 = 10
        viv1 = 0
        viv2 = 0
        x3 =0
        y3 =0
        
        ###########################
        
        
        if distance <= dmin2+0.35:  ##HAP
        
            hx1=translate(distance, dmin2, dmin2 + 0.35, 15, 60)    ##HAP
            hy1=-15                            
            stimuli = "position_35"
        
        '''
        if distance <= dmin2+0.3:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.3, 15, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_30"
            
        if distance <= dmin2+0.2:  ##HAP
        
            hx1=translate(distance,dmin2, dmin2 + 0.2, 15, 60)    ##HAP
            hy1=-13                            
            stimuli = "position_20"
        
        '''
        #######################     
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
        print('Actual Point',P)
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('bt.json', 'r') as fa:
                data = json.load(fa)
                n_points = data['general']['n']
                action = data['position'+str(n_counter)]['action']
                if action == "open":
                    rob.speedl((0 , 0, 0 , 0, 0, 0), acc=0.3, min_time=1);
                    time.sleep(0.1)
                    robotiqgrip.open_gripper()
                    time.sleep(0.1)
                elif action == "close":
                    rob.speedl((0 , 0, 0 , 0, 0, 0), acc=0.3, min_time=1);
                    time.sleep(0.1)
                    robotiqgrip.close_gripper()
                    time.sleep(0.1)
                elif action == "end":
                    rob.close()
                    hap.close()
                    sys.exit()
                    break
                else:
                    x_refr = data['position'+str(n_counter)]['xref']
                    y_refr = data['position'+str(n_counter)]['yref']
                    z_refr = data['position'+str(n_counter)]['zref']

        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin and n_r_ao > dmin2:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            
            viv1 = 0    #### HAP 
            viv2 = 30    #### HAP
            stimuli = "collision_one" #HAP
            
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            
            if  teta < np.pi*kangle/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*hv[0,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*hv[1,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*hv[2,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[2,0]
        
        elif n_r_ao < dmin2:
            vx1 = 0
            vy1 = 0
            vz1 = 0
            
            viv1 = 70    #### HAP 
            viv2 = 70    #### HAP
            stimuli = "collision_one"
            hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))

            rob.set_freedrive(1)
            
            while n_r_ao < dmin2+0.1:
                
                f1w.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
                
                try:
                    xo_refr = 0.43 * -df[0]  + 0.9  * df[2] + 1.297 +0.045 #+ hand
                    yo_refr = 0.9  *  -df[0] - 0.43 * df[2] - 0.183 -0.031 #+ hand
                    zo_refr = df[1] - 0.794 + 0.0006 #+ hand
                    print("Position of the ALT with respect to the center of the UR base frame is : " + str(xo_refr) +' ,'+ str(yo_refr) +' ,' + str(zo_refr))

                except:
                    xo_refr = 0
                    yo_refr = 0
                    zo_refr = 0

                t = rob.get_pose()
              #  print("Transformation from base to tcp is: ", t)


                pose = rob.getl()
                xn = float(pose[0])
                yn = float(pose[1])
                zn = float(pose[2])

                P = np.zeros((3,1));
                P[0,0] = float(xn)
                P[1,0] = float(yn)
                P[2,0] = float(zn)



                P_obs = np.zeros((3,1));   
                P_obs[0,0] = float(xo_refr)
                P_obs[1,0] = float(yo_refr)
                P_obs[2,0] = float(zo_refr)

                r_ao = P_obs - P
                n_r_ao = np.linalg.norm(r_ao)
                rob.set_freedrive(1)
                
                
            rob.set_freedrive(0)
        
        
        else :
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            #viv1=0
            hap.write(bytes(str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3)+ "\n",'utf-8'))
            print("sent to the hap: " + str(hx1) + "," + str(hy1)+ "," + str(viv1)+ "," + str(viv2)+ "," + str(x3)+ "," + str(y3))
            #####          
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=1);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        #print(e)
        f1w.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+ ";"+str(stimuli)+"\n")
           
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')